# **Electricity Price Forecasting**

**Author:** Milos Saric [https://saricmilos.com/]  
**Date:** December 10, 2025 - December 18th, 2025  
**Dataset:** Red Eléctrica — *[Spanish Electricity Market](https://www.ree.es/en/datos/generation)*  

---

This notebook explores Spanish electricity market datasets to build predictive models for forecasting tomorrow’s electricity prices using **machine learning** techniques. 
 
**Objective:**  
To develop an accurate **electricity price forecasting model** capable of anticipating day-ahead prices, helping energy producers, traders, and consumers make informed decisions.
The analysis will guide you through the complete data science workflow, including:

1. **Problem Definition** – Define the forecasting goal: predicting next-day electricity prices using historical data, demand patterns, and external factors (e.g., weather, holidays). Establish evaluation metrics such as **MAE, RMSE, MAPE**.  

2. **Data Collection & Overview** – Load and inspect datasets containing historical electricity prices, generation, demand, and weather information. Understand the structure, relationships, and key features.  

3. **Exploratory Data Analysis (EDA)** – Analyze price trends, daily/weekly patterns, seasonal effects, and correlations between electricity demand, weather, and prices. Visualize insights with plots.

4. **Data Preprocessing & Feature Engineering** – Clean data, handle missing values, create lag features, rolling averages, and encode categorical variables. Normalize features for model input.  

5. **Model Development** –  
- **Machine Learning Models:** Linear Regression, Random Forest, Gradient Boosting for price prediction.  

6. **Evaluation & Testing** – Assess model performance using metrics like MAE, RMSE, and MAPE. Compare approaches, visualize predicted vs actual prices, and interpret model results.  

7. **Deployment & Future Work** – Discuss strategies for deploying the forecasting model (via FastAPI) and potential improvements with more features or hybrid models.
---

# **About Electricity Price Forecasting Systems**

Electricity prices fluctuate rapidly throughout the day, influenced by factors such as **demand**, **weather conditions**, **fuel costs**, and the availability of **renewable energy sources** like wind and solar. Predicting these prices is essential for ensuring stability, optimizing energy usage, reducing costs, and enabling smarter grid management.

In modern energy systems, accurate forecasting plays a key role in helping both providers and consumers make informed decisions. Energy companies rely on forecasts to determine when to buy or sell electricity, while consumers and businesses can adjust consumption to take advantage of cheaper prices or avoid expensive peak periods.  

Because electricity **CAN NOT be stored easily**, unlike fossil fuels (gas,oil,coal etc.) it must be produced (or bought) the moment it is needed. This means supply and demand factors have an **immediate** impact on price, making forecasting a challenging yet vital task.

Effective electricity price prediction has several important applications:

- **Optimizing electricity storage** and battery usage.  
- **Enabling demand-side flexibility**, helping buildings reduce consumption during expensive periods (and increase during low or negative price periods).  
- **Reducing carbon emissions**, since electricity price often correlates with carbon intensity of generation.  
- **Supporting grid stability**, reducing strain during peak hours and balancing supply and demand efficiently.  

---

## **1. Problem Definition**

The first step is to establish a clear understanding of the forecasting challenge. This creates the foundation for the entire project and ensures all further work stays aligned with the primary objective.

### **Objective**
Develop an **electricity price forecasting system** that predicts **day-ahead electricity prices** using historical price data, generation and demand patterns, weather variables, and other relevant features.  
The system should generate accurate forecasts that support smarter decision-making for energy providers, grid operators, and end users.

### **Scope**
The analysis focuses on electricity market data, which includes:

- **Historical electricity prices** — day-ahead and intraday market values.  
- **Energy generation data** — renewable and non-renewable production levels.  
- **Weather data** — temperature, solar irradiance, wind speed, etc.  
- **Demand/load data** — total electricity consumption patterns.  

Predictions will be based on the dataset provided, with optional integration of external weather or market data in advanced stages.

### **Stakeholders**
- **Energy Providers / Traders:** Optimize buying and selling strategies to reduce risk and increase profitability.  
- **Grid Operators:** Improve load balancing and ensure efficient grid stability.  
- **Businesses & Industries:** Adjust operational schedules to lower electricity costs.  
- **Consumers:** Reduce electricity bills through smarter usage patterns.  
- **Data Scientists / ML Engineers:** Explore forecasting algorithms and improve model accuracy.  

### **Success Criteria**
A successful forecasting system should produce **accurate, reliable, and robust predictions**, assessed using metrics such as:

- **RMSE** — Root Mean Squared Error  
- **MAE** — Mean Absolute Error  
- **MAPE** — Mean Absolute Percentage Error  

These metrics measure how close the predicted prices are to real future prices.

> A clearly defined problem enables the development of a powerful and effective electricity price forecasting model that supports smarter, greener, and more efficient energy systems.

## **2. Data Collection**

The **Data Collection** phase focuses on gathering and preparing the datasets required to build and evaluate the electricity price forecasting models. This step also involves importing essential libraries, setting up the working environment, and organizing reusable functions to ensure a smooth analysis workflow.

## **Dataset Description**



---

We can begin by:
1. Loading each dataset (`energy_dataset`, `weather_features`,) individually.  
2. Performing exploratory data analysis (EDA) to understand distributions and missing values.  
3. Merging the datasets to form a unified view of user–book interactions.  
4. Building and evaluating different recommendation approaches. 

# Required Libraries Import

### Setting working paths

In [3]:
import sys
from pathlib import Path

In [4]:
# Go up one level from /notebooks to the main folder
project_root = Path.cwd().parent

In [5]:
sys.path.append(str(project_root))

### Core Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Modules

In [2]:
%load_ext autoreload
%autoreload 2

# Modules
from src.dataloader import load_all_csvs_from_folder

ModuleNotFoundError: No module named 'src'

In [8]:
# Build the path to the data file
data_path = project_root / "data"


In [9]:
datasets = load_all_csvs_from_folder(data_path,low_memory = False)

In [10]:
print(f"{datasets.keys()}")

dict_keys(['energy_dataset', 'weather_features'])


In [11]:
energy_df, weather_df = (datasets.get(key) for key in ["energy_dataset","weather_features"])

In [14]:
energy_df.head(50)

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04
5,2015-01-01 05:00:00+01:00,410.0,178.0,0.0,4038.0,3590.0,156.0,0.0,0.0,0.0,...,188.0,0.0,4618.0,4.0,NaN,4617.0,21441.0,19905.0,35.72,53.63
6,2015-01-01 06:00:00+01:00,401.0,172.0,0.0,4040.0,3368.0,158.0,0.0,0.0,0.0,...,186.0,0.0,4397.0,3.0,NaN,4276.0,21285.0,20010.0,35.13,51.73
7,2015-01-01 07:00:00+01:00,408.0,172.0,0.0,4030.0,3208.0,160.0,0.0,0.0,0.0,...,189.0,0.0,3992.0,12.0,NaN,3994.0,21545.0,20377.0,36.22,51.43
8,2015-01-01 08:00:00+01:00,413.0,177.0,0.0,4052.0,3335.0,161.0,0.0,0.0,0.0,...,198.0,0.0,3629.0,39.0,NaN,3602.0,21443.0,20094.0,32.40,48.98
9,2015-01-01 09:00:00+01:00,419.0,177.0,0.0,4137.0,3437.0,163.0,0.0,0.0,0.0,...,198.0,0.0,3073.0,784.0,NaN,3212.0,21560.0,20637.0,36.60,54.20


In [13]:
energy_df.columns

Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage aggregated',
       'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind offshore eday ahead', 'forecast wind onshore day ahead',
       'total load forecast', 'total load actual', 'price day ahead',
       'price actual'],
      dtype='object')